In [73]:
from fractions import Fraction
from scipy.special import lambertw
from math import gcd

def normalize_fraction(frac, max_bits=256):
    """ Normalize the fraction so both numerator and denominator fit within <max_bits>. """
    max_val = 2 ** (max_bits - 1) - 1
    num, denom = frac.numerator, frac.denominator
    while abs(num) > max_val or abs(denom) > max_val:
        # Scale down the fraction by dividing both parts by 2
        num, denom = num // 2, denom // 2
    
    return Fraction(num, denom)

def fits_precision(term, max_bits=256):
    """Check if the term is smaller than what can be represented in the specified number of bits."""
    # Calculate the smallest representable value for a given bit size
    min_value = 1 / (2 ** (max_bits - 1))
    return abs(term) < min_value

def log(x, max_bits=256):
    """Compute the natural logarithm of x using a series expansion."""
    if x <= 0:
        raise ValueError("Logarithm is undefined for non-positive values.")
    
    y = (x - Fraction(1)) / (x + Fraction(1))
    result = Fraction(0)
    n = 1
    
    while True:
        term = ((-1) ** (n + 1)) * (y ** n) / n
        new_result = result + term
        
        # Normalize the result
        new_result = normalize_fraction(new_result)
        
        if fits_precision(term, max_bits):
            break  # Stop if the term is too small to affect precision within the bit limit
        
        result = new_result
        n += 1
    
    return result * 2

def exp(x, max_bits=256):
    """Compute e^x using a series expansion."""
    result = Fraction(1)
    term = Fraction(1)
    n = 1
    
    while True:
        term *= x / n
        new_result = result + term
        
        # Normalize the result
        new_result = normalize_fraction(new_result)
        
        if fits_precision(term, max_bits):
            break  # Stop if the term is too small to affect precision within the bit limit
        
        result = new_result
        n += 1
    
    return result

def lambert_w_fraction(x, max_bits=256):
    """Compute Lambert W function for x using Newton's method."""
    if x == 0:
        return Fraction(0)
    
    # Initial guess
    w = Fraction(lambertw(float(x)).real)
    
    while True:
        e_w = exp(w, max_bits)
        f_w = w * e_w - x
        f_prime_w = e_w * (w + 1)
        
        # Newton's method update: w = w - f(w) / f'(w)
        new_w = w - f_w / f_prime_w
        
        # Normalize the result
        new_w = normalize_fraction(new_w)
        
        if fits_precision(f_w / f_prime_w, max_bits):
            break  # Stop if the correction term is too small to affect precision within the bit limit
        
        w = new_w
    
    return w

In [74]:
log(Fraction(1,2))

Fraction(-22165288075754041349619509952434724978441066390835170094929633039240746399401, 27333163362901613601132434115890135952555425666077976971589371399160323879670)

In [75]:
w17 = lambert_w_fraction(Fraction(1,7), 256)
print(f"w(1/7) = {float(w17)} scipy.special.lambertw(1/7) = {lambertw(1/7).real}")

w(1/7) = 0.12595113283617557 scipy.special.lambertw(1/7) = 0.12595113283617557


In [72]:
Fraction(lambertw(1/7).real)

Fraction(4537867799263137, 36028797018963968)

In [76]:
TARGET_NC = 7
for nb in range(TARGET_NC, 40):
    w = lambert_w_fraction(Fraction(nb,7), 256)
    print(f"W({nb}/7-1) = {float(w)}   {lambertw(nb/TARGET_NC).real}")

W(7/7-1) = 0.5671432904097838   0.5671432904097838
W(8/7-1) = 0.6167778161075422   0.6167778161075422
W(9/7-1) = 0.6627188557116432   0.6627188557116432
W(10/7-1) = 0.7055097010790102   0.7055097010790102
W(11/7-1) = 0.7455791302671205   0.7455791302671206
W(12/7-1) = 0.7832718955427015   0.7832718955427014
W(13/7-1) = 0.8188696156542401   0.8188696156542402
W(14/7-1) = 0.8526055020137255   0.8526055020137254
W(15/7-1) = 0.8846749930084885   0.8846749930084885
W(16/7-1) = 0.915243596566803   0.915243596566803
W(17/7-1) = 0.944452781455909   0.944452781455909
W(18/7-1) = 0.972424475500165   0.972424475500165
W(19/7-1) = 0.9992645503434486   0.9992645503434486
W(20/7-1) = 1.025065556445599   1.025065556445599
W(21/7-1) = 1.04990889496404   1.04990889496404
W(22/7-1) = 1.0738665609038538   1.0738665609038538
W(23/7-1) = 1.0970025557868635   1.0970025557868635
W(24/7-1) = 1.1193740426839116   1.1193740426839116
W(25/7-1) = 1.1410322983118264   1.1410322983118266
W(26/7-1) = 1.1620235037562

In [43]:
(float(_) - lambertw(1/7).real) / float(_)

np.float64(0.0021054829516762874)

In [29]:
fit_within_bits(lambert_w_fraction(Fraction(1,7), 1), max_bits=256)

Fraction(52615563062464827257207977029413699224599281681447, 415036611773027758665107587226080964835247810150400)

In [8]:
%timeit exp(Fraction(1,7), 10)

28.2 μs ± 1.08 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%timeit log(Fraction(1,7), 10)

44.3 μs ± 971 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
lambert_w_fraction(Fraction(1,7), 10)

KeyboardInterrupt: 